<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Product_Marketing_AI_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Marketing AI System

## Overview
This system helps create high-quality marketing images automatically. It takes in photos and optional audio or video, then processes, refines, and enhances them to produce beautiful marketing visuals for many different industries.

## Key Features

- **Easy Input:** Upload main and supplementary images, plus optional multimedia for extra context.
- **Smart Processing:** The system automatically cuts out key parts, improves image details, and boosts overall clarity.
- **Creative Prompts:** Custom prompts are generated to guide the image creation process, making it tailored to your needs.
- **Fast Generation:** Uses multiple AI models working together to generate and improve images quickly.
- **Quality Check:** Compares final images to the originals and provides simple quality feedback.
- **Simple Reports:** Automatically produces a brief report with the final prompt and quality scores.

## Benefits
- Saves time by automating the creation of professional marketing images.
- Provides consistent and attractive visuals optimized for your business.
- Easy to use with straightforward input and clear feedback.

Enjoy a seamless experience in making your marketing visuals stand out!

In [5]:
# Step 1: Install dependencies (for Colab Notebook; not needed if pre-installed)
%pip install --q git+https://github.com/facebookresearch/segment-anything.git
%pip install -q diffusers transformers accelerate
%pip install -q git+https://github.com/ChaoningZhang/Real-ESRGAN.git
%pip install -q git+https://github.com/advimman/lama.git
%pip install -q langchain langchain-google-genai groq openai u2net langchain_community
%pip install -q scikit-image
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O sam_vit_h.pth -q

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/Real-ESRGAN.git /tmp/pip-req-build-54h02w0u did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/ChaoningZhang/Real-ESRGAN.git /tmp/pip-req-build-54h02w0u did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
ERROR: git+https://github.com/advimman/lama.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
ERROR: Could not find a version that satisfies the requirement u2net (from versions: none)
ERROR: No matching distribution found for u2net


In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import base64
import json
import hashlib
import asyncio
import concurrent.futures
from tenacity import retry, stop_after_attempt, wait_exponential
from skimage.metrics import structural_similarity as ssim
import math

# Simplified LLM import from LangChain (using only text-generation model)
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

print("Starting Product Marketing AI Pipeline with advanced parallelization and async/await...")

# Create outputs directory for intermediate and final outputs
OUTPUT_DIR = "outputs"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Utility Functions
def free_memory(model):
    try:
        del model
        import torch
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"Error in memory cleanup: {e}")

def verify_checksum(file_path, expected_hash):
    sha256 = hashlib.sha256()
    with open(file_path, "rb") as f:
        sha256.update(f.read())
    return sha256.hexdigest() == expected_hash

def psnr(target, ref):
    """Compute Peak Signal-to-Noise Ratio between two images."""
    mse = np.mean((target - ref) ** 2)
    if mse == 0:
        return 100
    max_pixel = 255.0
    return 20 * math.log10(max_pixel / math.sqrt(mse))

In [6]:
# Step 3: Initialize Models
print("Initializing models...")

# Load SAM
from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint="sam_vit_h.pth")
# Replace "expected_hash" with actual expected checksum for production
if not verify_checksum("sam_vit_h.pth", "expected_hash"):
    raise ValueError("SAM checkpoint corrupted")
predictor = SamPredictor(sam)

# Load Stable Diffusion with ControlNet fallback
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_seg",
    torch_dtype="float16",
    local_files_only=False
)
sd_pipeline = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype="float16"
).to("cuda")

# Load ESRGAN for super-resolution
from RealESRGAN import RealESRGAN
esrgan_model = RealESRGAN(device="cuda", model="RealESRGAN_x4plus")
esrgan_model.load_weights("RealESRGAN_x4plus.pth")

# Load LaMa for inpainting
from lama import LaMa
lama_model = LaMa(device="cuda")

# Load U2-Net for object extraction
from u2net import U2NET
u2net = U2NET()

# Initialize Groq client and Google Gemini
from groq import Groq
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
from langchain_google_genai import ChatGoogleGenerativeAI
gemini_llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Initialize xAI client
from openai import OpenAI
xai_client = OpenAI(base_url="https://api.x.ai/v1", api_key=os.environ.get("XAI_API_KEY"))

# Setup LangChain for prompt engineering using a basic text-generation model
text_generator = pipeline("text-generation", model="gpt2", device=0)
llm = HuggingFacePipeline(pipeline=text_generator)

Initializing models...


NameError: name 'verify_checksum' is not defined